In [6]:
import json
from datetime import datetime


# filename = './../corona-out-2'
filename = './../corona-out-3'

In [7]:
def to_timestamp(val):
    dt = val.split(' ')
    st = f"{dt[5]}-{dt[1]}-{dt[2]} {dt[3]}"
    dt = datetime.strptime(st, "%Y-%b-%d %H:%M:%S")
    return int(dt.timestamp())

def to_dtstring(val):
    return datetime.fromtimestamp(val).strftime('%b %d, %Y - %X')

def get_hashtags(data):
    hashtags = []
    for item in data:
        hashtags.append(item['text'])
    return hashtags

In [8]:
# def add_user(user):
#     if (user['id'] not in users):
#         users[user['id']] = user

# def add_tweet(tweet, tweet_type):
#     if tweet_type=='tweet':
#         if tweet['id'] not in tweets:
#             tweets[tweet['id']] = tweet
#     elif tweet_type=='retweet':
#         if tweet['id'] not in retweets:
#             retweets[tweet['id']] = tweet

# def add_retweet(retweet):
#     if (retweet['id'] not in retweets):
#         retweets[retweet['id']] = retweet

def add_user(user):
    if (user['id'] not in users):
        new_user = {}
        
        new_user['id'] = user['id']
        new_user['name'] = user['name']
        new_user['screen_name'] = user['screen_name']
        new_user['location'] = user['location']
        new_user['url'] = user['url']
        new_user['description'] = user['description']
        new_user['verified'] = user['verified']
        new_user['followers_count'] = user['followers_count']
        new_user['friends_count'] = user['friends_count']
        new_user['listed_count'] = user['listed_count']
        new_user['favourites_count'] = user['favourites_count']
        new_user['statuses_count'] = user['statuses_count']
        new_user['created_at'] = to_timestamp(user['created_at'])
        
        users[user['id']] = new_user


def process_tweet(tweet):
    new_tweet = {}
        
    new_tweet['created_at'] = to_timestamp(tweet['created_at'])
    new_tweet['id'] = tweet['id']
    if tweet['truncated']:
        new_tweet['text'] = tweet['extended_tweet']['full_text']
        new_tweet['hashtags'] = get_hashtags(tweet['extended_tweet']['entities']['hashtags'])
    else:
        new_tweet['text'] = tweet['text']
        new_tweet['hashtags'] = get_hashtags(tweet['entities']['hashtags'])

    user = {}
    user['id'] = tweet['user']['id']
    user['name'] = tweet['user']['name']
    user['screen_name'] = tweet['user']['screen_name']
    user['verified'] = tweet['user']['verified']
    new_tweet['user'] = user

    new_tweet['source'] = tweet['source']
    new_tweet['quote_count'] = tweet['quote_count']
    new_tweet['reply_count'] = tweet['reply_count']
    new_tweet['retweet_count'] = tweet['retweet_count']
    new_tweet['favorite_count'] = tweet['favorite_count']
    
    if 'retweeted_status' in tweet:
        new_tweet['retweeted_status'] = process_tweet(tweet['retweeted_status'])
        
    new_tweet['is_quote_status'] = tweet['is_quote_status']
    if tweet['is_quote_status'] and 'quoted_status' in tweet:
        new_tweet['quoted_status'] = process_tweet(tweet['quoted_status'])
    
    return new_tweet
        
def add_tweet(tweet, tweet_type):
    if (tweet['id'] not in tweets):
        
        new_tweet = process_tweet(tweet)
        
        if tweet_type=='tweet':
            tweets[tweet['id']] = new_tweet
        elif tweet_type=='retweet':
            retweets[tweet['id']] = new_tweet
        elif tweet_type=='quote':
            quotes[tweet['id']] = new_tweet

In [9]:
def process_data(data):
    if data['id'] in tweets:
        return
    
    add_user(data['user'])

    if data['text'].startswith('RT'):
        add_tweet(data, 'retweet')
        if 'retweeted_status' in data:
            process_data(data['retweeted_status'])

    else:
        add_tweet(data, 'tweet')

In [10]:
users = {}
tweets = {}
retweets = {}
quotes = {}

for line in open(filename, 'r'):
    if len(line.strip()) == 0:
        continue
        
#     try:
    data = json.loads(line)
    process_data(data)

#     except:
#         continue

In [11]:
# (7349, 11157, 15505)
# (10987, 11157, 18461)

len(tweets), len(retweets), len(users), len(quotes)

(50915, 61108, 88068, 0)

In [12]:
idxs = list(retweets.keys())

i = 0
for idx in idxs:
    if i==10: break
    print(retweets[idx]['text'])
    print(retweets[idx]['retweeted_status']['text'])
    print()
    i+=1

RT @BJP4India: India’s war with Corona is ongoing.

Play your part and make sure no one around you starves.

Your contribution to defeat hu…
India’s war with Corona is ongoing.

Play your part and make sure no one around you starves.

Your contribution to defeat hunger is as valuable as defeating Corona.

#FeedTheNeedy https://t.co/TUWf2nqPgZ

RT @schrodingerk42: @ozkan_yalim @DurmusYillmaz 
Açık kapalı görüşler yasak olduğu için sadece telefon görüşlerinde kendisinden haber alabi…
@ozkan_yalim @DurmusYillmaz 
Açık kapalı görüşler yasak olduğu için sadece telefon görüşlerinde kendisinden haber alabiliyoruz ancak ameliyat olduktan sonraki hafta telefon görüşünde kendisi aramadı.Corona endişesinin olduğu şu günlerde kendisinden haber alabilmek için haftasonu+

RT @MonstaXEurope: VIDEO | 25.04.20

&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona... Turning on the Music industry.…
VIDEO | 25.04.20

&gt; Monsta X appears on Yonhap News TV - Idol comeback through Corona

In [13]:
retweets[idxs[1]]

{'created_at': 1587831702,
 'id': 1254022772877131777,
 'text': 'RT @schrodingerk42: @ozkan_yalim @DurmusYillmaz \nAçık kapalı görüşler yasak olduğu için sadece telefon görüşlerinde kendisinden haber alabi…',
 'hashtags': [],
 'user': {'id': 1206650133976408064,
  'name': 'Büşra Öztaş',
  'screen_name': 'schrodingerk42',
  'verified': False},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'retweeted_status': {'created_at': 1587486840,
  'id': 1252576316135739392,
  'text': '@ozkan_yalim @DurmusYillmaz \nAçık kapalı görüşler yasak olduğu için sadece telefon görüşlerinde kendisinden haber alabiliyoruz ancak ameliyat olduktan sonraki hafta telefon görüşünde kendisi aramadı.Corona endişesinin olduğu şu günlerde kendisinden haber alabilmek için haftasonu+',
  'hashtags': [],
  'user': {'id': 1206650133976408064,
   'name': 'Büşra Öztaş',
   'screen_name': 'sc

In [14]:
retweets[idxs[0]].keys()

dict_keys(['created_at', 'id', 'text', 'hashtags', 'user', 'source', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'retweeted_status', 'is_quote_status'])

In [15]:
idxs = list(tweets.keys())

i = 0
for idx in idxs:
    if i==10: break
    print(tweets[idx])
    print()
    i+=1

{'created_at': 1587831701, 'id': 1254022770679320576, 'text': 'É isto, ou vou morrer sem ar ou com o corona https://t.co/O0Y7B3Koj4', 'hashtags': [], 'user': {'id': 804046791348015107, 'name': 'Bi Sex Uau', 'screen_name': 'B_King69', 'verified': False}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'is_quote_status': False}

{'created_at': 1587831701, 'id': 1254022770746372096, 'text': 'Schöne Runde mit dem Rennrad ✌️\n#sport #corona https://t.co/Uglj9YJPI1', 'hashtags': ['sport', 'corona'], 'user': {'id': 2242948745, 'name': 'Thomas Krause', 'screen_name': 'tho1965', 'verified': False}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'is_quote_status': False}

{'created_at': 1587814212, 'id': 1253949413191344128, 'text': 'India’s war w

In [16]:
tweets[idxs[0]]

{'created_at': 1587831701,
 'id': 1254022770679320576,
 'text': 'É isto, ou vou morrer sem ar ou com o corona https://t.co/O0Y7B3Koj4',
 'hashtags': [],
 'user': {'id': 804046791348015107,
  'name': 'Bi Sex Uau',
  'screen_name': 'B_King69',
  'verified': False},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'quote_count': 0,
 'reply_count': 0,
 'retweet_count': 0,
 'favorite_count': 0,
 'is_quote_status': False}

In [17]:
tweets[idxs[1]].keys()

dict_keys(['created_at', 'id', 'text', 'hashtags', 'user', 'source', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'is_quote_status'])